In [6]:
import os
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForQuestionAnswering, TrainingArguments, Trainer

In [2]:
df = pd.read_csv("202302_groundtruth.csv")
df.head()

,Serial,ID,Year,Date,Title,Violation,Court_location,Link,Text,犯罪行為,被告姓名,被告年紀,被告性別,被告職業,被告學歷,被告經濟狀況
0,1,"KMDM,111,原易,1,20230224,2",111,20230224,詐欺,"刑法第30條, 為洗錢防制法第14條, 依刑事訴訟法第451條, 錢防制法第14條, 則應回...",福建金門地方法院,https://data.judicial.gov.tw/opendl/JDocFile/K...,福建金門地方法院刑事判決 \n111年度原易字第1號\n公 訴 人 福建金門地方檢察署...,提供帳戶給詐騙集團，詐騙集團進行網路交友、投資詐欺，被告提領詐得款項。,王允文,21,男,板模工人,國中畢業,一般，月收入約為4萬元
1,2,"KMDM,111,金訴,18,20230218,1",111,20230218,詐欺等,"不符合刑法第74條, 刑法第51條, 併依刑法第74條, 依刑法第57條, 與憲法第8條, ...",福建金門地方法院,https://data.judicial.gov.tw/opendl/JDocFile/K...,福建金門地方法院刑事判決 \n111年度金訴字第22號\n111年度金訴字第23號\n111...,提供金融帳戶給詐騙集團使用，詐騙集團進行詐騙，被告提領詐得款項。,"韓銘泓, 李宜婷, 陳立揚",判決書中未明確提及,"韓銘泓：男, 李宜婷：女, 陳立揚：男","待業中, 會計, 飯店櫃檯","韓銘泓：大學畢業, 李宜婷：高職肄業, 陳立揚：國中畢業","韓銘泓：已婚無子，與妻子同住，需扶養祖父母，靠之前存款為生, 李宜婷：未婚無子女，與父母同住..."
2,3,"NTDM,111,原易,15,20230202,1",111,20230202,詐欺,"係犯刑法第30條, 中華民國刑法第339條, 依照刑法第30條, 依刑事訴訟法第299條, ...",臺灣南投地方法院,https://data.judicial.gov.tw/opendl/JDocFile/N...,臺灣南投地方法院刑事判決\n111年度原易字第15號\n公 訴 人 臺灣南投地方檢察署...,提供金融卡及密碼給詐騙集團，詐騙集團詐騙被害人匯款，並提領款項。,徐夢萍,判決書中未明確提及,女,種香菇太空包工作,高中肄業,照顧孫子，偶爾打零工，配偶領有殘障補助
3,4,"NTDM,111,審原訴,21,20230209,1",111,20230209,詐欺,"係犯刑法第30條, ㈠按刑法第339條, 中華民國刑法第339條, 依照刑法第30條, 依刑...",臺灣南投地方法院,https://data.judicial.gov.tw/opendl/JDocFile/N...,臺灣南投地方法院刑事判決\n111年度審原訴字第21號\n公 訴 人 臺灣南投地方檢察...,提供申辦的行動電話門號給詐騙集團，詐騙集團註冊帳號並進行詐騙。,江正生,判決書中未明確提及,男,在菜市場打工,大學肄業,無收入，與父母同住，等待當兵
4,5,"NTDM,111,審易,164,20230223,1",111,20230223,詐欺,"依刑事訴訟法第273條, 是均依刑法第38條, 中華民國刑法第339條, 係犯刑法第339條",臺灣南投地方法院,https://data.judicial.gov.tw/opendl/JDocFile/N...,臺灣南投地方法院刑事判決\n 111年度審易字第164號\n公 訴 人 臺灣南投地方...,假借投資擴大營業，向他人借款後失聯，無法兌現本票。,陳敬和,判決書中未明確提及,男,鋼鐵承包商,高中,沒有家屬要扶養


In [3]:
df["Date"] = pd.to_datetime(df["Date"], format="%Y%m%d")
df['Date'] = df['Date'].dt.date

clear_df = df.copy()

#去除本次不會用到的欄位
drop_cols = ['Serial', 'ID', 'Date', 'Link', 'Violation', 'Year', 'Title', 'Court_location']
clear_df.drop(drop_cols, axis = 1, inplace = True)

#去除文章內容為空值的筆數
clear_df.dropna(subset = ['Text'], axis=0, how='any', inplace=True)

#移除'\n\n'，並移除'\n'
clear_df['Text'] = clear_df['Text'].str.replace(r'\n\n\n','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'\n\n','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'\n','', regex=True)

#移除'\u3000'
clear_df['Text'] = clear_df['Text'].str.replace(r'\u3000','', regex=True)

#移除多餘空格
clear_df['Text'] = clear_df['Text'].str.replace(r' ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'  ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'   ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'    ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'     ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'      ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'       ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'        ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'         ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'          ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'           ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'            ','', regex=True)
clear_df['Text'] = clear_df['Text'].str.replace(r'             ','', regex=True)
clear_df.head()

,Text,犯罪行為,被告姓名,被告年紀,被告性別,被告職業,被告學歷,被告經濟狀況
0,福建金門地方法院刑事判決111年度原易字第1號公訴人福建金門地方檢察署檢察官被告王允文指定辯...,提供帳戶給詐騙集團，詐騙集團進行網路交友、投資詐欺，被告提領詐得款項。,王允文,21,男,板模工人,國中畢業,一般，月收入約為4萬元
1,福建金門地方法院刑事判決111年度金訴字第22號111年度金訴字第23號111年度金訴字第2...,提供金融帳戶給詐騙集團使用，詐騙集團進行詐騙，被告提領詐得款項。,"韓銘泓, 李宜婷, 陳立揚",判決書中未明確提及,"韓銘泓：男, 李宜婷：女, 陳立揚：男","待業中, 會計, 飯店櫃檯","韓銘泓：大學畢業, 李宜婷：高職肄業, 陳立揚：國中畢業","韓銘泓：已婚無子，與妻子同住，需扶養祖父母，靠之前存款為生, 李宜婷：未婚無子女，與父母同住..."
2,臺灣南投地方法院刑事判決111年度原易字第15號公訴人臺灣南投地方檢察署檢察官被告徐夢萍指定...,提供金融卡及密碼給詐騙集團，詐騙集團詐騙被害人匯款，並提領款項。,徐夢萍,判決書中未明確提及,女,種香菇太空包工作,高中肄業,照顧孫子，偶爾打零工，配偶領有殘障補助
3,臺灣南投地方法院刑事判決111年度審原訴字第21號公訴人臺灣南投地方檢察署檢察官被告江正生指...,提供申辦的行動電話門號給詐騙集團，詐騙集團註冊帳號並進行詐騙。,江正生,判決書中未明確提及,男,在菜市場打工,大學肄業,無收入，與父母同住，等待當兵
4,臺灣南投地方法院刑事判決111年度審易字第164號公訴人臺灣南投地方檢察署檢察官被告陳敬和上...,假借投資擴大營業，向他人借款後失聯，無法兌現本票。,陳敬和,判決書中未明確提及,男,鋼鐵承包商,高中,沒有家屬要扶養


In [4]:
clear_df = clear_df.iloc[:78]
clear_df

,Text,犯罪行為,被告姓名,被告年紀,被告性別,被告職業,被告學歷,被告經濟狀況
0,福建金門地方法院刑事判決111年度原易字第1號公訴人福建金門地方檢察署檢察官被告王允文指定辯...,提供帳戶給詐騙集團，詐騙集團進行網路交友、投資詐欺，被告提領詐得款項。,王允文,21,男,板模工人,國中畢業,一般，月收入約為4萬元
1,福建金門地方法院刑事判決111年度金訴字第22號111年度金訴字第23號111年度金訴字第2...,提供金融帳戶給詐騙集團使用，詐騙集團進行詐騙，被告提領詐得款項。,"韓銘泓, 李宜婷, 陳立揚",判決書中未明確提及,"韓銘泓：男, 李宜婷：女, 陳立揚：男","待業中, 會計, 飯店櫃檯","韓銘泓：大學畢業, 李宜婷：高職肄業, 陳立揚：國中畢業","韓銘泓：已婚無子，與妻子同住，需扶養祖父母，靠之前存款為生, 李宜婷：未婚無子女，與父母同住..."
2,臺灣南投地方法院刑事判決111年度原易字第15號公訴人臺灣南投地方檢察署檢察官被告徐夢萍指定...,提供金融卡及密碼給詐騙集團，詐騙集團詐騙被害人匯款，並提領款項。,徐夢萍,判決書中未明確提及,女,種香菇太空包工作,高中肄業,照顧孫子，偶爾打零工，配偶領有殘障補助
3,臺灣南投地方法院刑事判決111年度審原訴字第21號公訴人臺灣南投地方檢察署檢察官被告江正生指...,提供申辦的行動電話門號給詐騙集團，詐騙集團註冊帳號並進行詐騙。,江正生,判決書中未明確提及,男,在菜市場打工,大學肄業,無收入，與父母同住，等待當兵
4,臺灣南投地方法院刑事判決111年度審易字第164號公訴人臺灣南投地方檢察署檢察官被告陳敬和上...,假借投資擴大營業，向他人借款後失聯，無法兌現本票。,陳敬和,判決書中未明確提及,男,鋼鐵承包商,高中,沒有家屬要扶養
...,...,...,...,...,...,...,...,...
74,臺灣士林地方法院刑事判決111年度金訴字第470號公訴人臺灣士林地方檢察署檢察官被告鄭博元上...,被告鄭博元與李俊鵬、陳志豪及其他詐欺集團成員，共同假冒東森購物客服人員、主管及華南銀行客服人...,鄭博元,判決書中未明確提及,男,判決書中未明確提及,判決書中未明確提及,判決書中未明確提及
75,臺灣士林地方法院刑事判決111年度金訴字第557號公訴人臺灣士林地方檢察署檢察官被告葉平舞上...,被告葉平舞因玩「九州娛樂城」線上博奕而結識成年友人，葉平舞提供金融帳戶、提領轉交款項，並參與...,葉平舞,34歲,男,中古車買賣,高職肄業,未婚，育有1名未成年子女，目前由女友照顧
76,臺灣士林地方法院刑事判決111年度金訴字第306號111年度金訴字第307號111年度金訴字...,被告林育佑參與詐欺集團，假冒政府機關或公務員名義，對多名被害人進行詐騙，誘導被害人匯款至指定...,林育佑,判決書中未明確提及,男,美髮師及調酒師,大學肄業,未婚，需扶養重病之父親，月薪約新臺幣6至7萬元
77,臺灣士林地方法院刑事判決111年度金訴字第568號公訴人臺灣士林地方檢察署檢察官被告曾裕閔上...,被告林育佑參與詐欺集團，假冒政府機關或公務員名義，對多名被害人進行詐騙，誘導被害人匯款至指定...,林育佑,判決書中未明確提及,男,美髮師及調酒師,大學肄業,未婚，需扶養重病之父親，月薪約新臺幣6至7萬元


In [5]:
# 定義問題與對應的欄位
questions_dict = {
    "犯罪行為": "犯罪行為",
    "被告姓名": "被告姓名",
    "被告年紀": "被告年紀",
    "被告性別": "被告性別",
    "被告職業": "被告職業",
    "被告學歷": "被告學歷",
    "被告經濟狀況": "被告經濟狀況"
}

# 構建新的數據框
new_data = {
    "Text": [],
    "question": [],
    "answer": []
}

In [6]:
# 轉換數據
for index, row in clear_df.iterrows():
    for question, col in questions_dict.items():
        new_data["Text"].append(row["Text"])
        new_data["question"].append(question)
        new_data["answer"].append(row[col])

new_df = pd.DataFrame(new_data)

# 檢查新的數據框
new_df

,Text,question,answer
0,福建金門地方法院刑事判決111年度原易字第1號公訴人福建金門地方檢察署檢察官被告王允文指定辯...,犯罪行為,提供帳戶給詐騙集團，詐騙集團進行網路交友、投資詐欺，被告提領詐得款項。
1,福建金門地方法院刑事判決111年度原易字第1號公訴人福建金門地方檢察署檢察官被告王允文指定辯...,被告姓名,王允文
2,福建金門地方法院刑事判決111年度原易字第1號公訴人福建金門地方檢察署檢察官被告王允文指定辯...,被告年紀,21
3,福建金門地方法院刑事判決111年度原易字第1號公訴人福建金門地方檢察署檢察官被告王允文指定辯...,被告性別,男
4,福建金門地方法院刑事判決111年度原易字第1號公訴人福建金門地方檢察署檢察官被告王允文指定辯...,被告職業,板模工人
...,...,...,...
541,臺灣士林地方法院刑事判決111年度金訴字第580號111年度金訴字第581號111年度金訴字...,被告年紀,判決書中未明確提及
542,臺灣士林地方法院刑事判決111年度金訴字第580號111年度金訴字第581號111年度金訴字...,被告性別,男
543,臺灣士林地方法院刑事判決111年度金訴字第580號111年度金訴字第581號111年度金訴字...,被告職業,判決書中未明確提及
544,臺灣士林地方法院刑事判決111年度金訴字第580號111年度金訴字第581號111年度金訴字...,被告學歷,判決書中未明確提及


In [10]:
print(os.getcwd())

/mnt/sda/home/nsysu_eric/Eric


In [33]:
# 保存轉換後的數據框
#new_df.to_csv('finetune_dataset.csv', index=False)

In [7]:
# 構建數據集
data = {
    'context': new_df['Text'].tolist(),
    'question': new_df['question'].tolist(),
    'answers': [{'text': [ans], 'answer_start': [ctx.find(ans)]} for ctx, ans in zip(new_df['Text'], new_df['answer'])]
}

dataset = Dataset.from_dict(data)

In [8]:
# 加載模型和tokenizer
model_name = "chinese_pretrain_mrc_roberta_wwm_ext_large" # "chinese_pretrain_mrc_macbert_large"

# Use in Transformers
tokenizer = AutoTokenizer.from_pretrained(f"luhua/{model_name}")
model = AutoModelForQuestionAnswering.from_pretrained(f"luhua/{model_name}")

Some weights of the model checkpoint at luhua/chinese_pretrain_mrc_roberta_wwm_ext_large were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
import torch

# 釋放不需要的內存
torch.cuda.empty_cache()

In [10]:
# Tokenize function
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    inputs["start_positions"] = [ans["answer_start"][0] for ans in examples["answers"]]
    inputs["end_positions"] = [start + len(ans["text"][0]) for start, ans in zip(inputs["start_positions"], examples["answers"])]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|████████████████████████████████████████████████████████████████████| 546/546 [00:00<00:00, 968.93 examples/s]


In [11]:
# 設置訓練參數
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# 訓練
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

trainer.train()

/mnt/sda/home/nsysu_eric/.conda/envs/python39/lib/python3.9/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,No log,nan
2,No log,nan
3,No log,nan


TrainOutput(global_step=411, training_loss=1.8013926410907086, metrics={'train_runtime': 81.1046, 'train_samples_per_second': 20.196, 'train_steps_per_second': 5.068, 'total_flos': 1521222053105664.0, 'train_loss': 1.8013926410907086, 'epoch': 3.0})

In [12]:
# 保存模型
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')